Setup & Keys

In [73]:
# Cell 1: Setup
from dotenv import load_dotenv
import os

load_dotenv()  # lädt .env

BASE_URL = "https://api.cerebras.ai/v1"
LLM_MODEL = "gpt-oss-120b"
LLM_TEMPERATURE = 0.3
LLM_API_KEY = os.environ["CEREBRAS_API_KEY"]


LLM

In [74]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url=BASE_URL,
    api_key=LLM_API_KEY,
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
)

Webseiten laden (gezielte URLs + User-Agent)

In [75]:
# Cell 3: Webseiten laden
import os
os.environ["USER_AGENT"] = "Mozilla/5.0 (MM-FAQ-Agent/1.0)"

from langchain_community.document_loaders import WebBaseLoader

FAQ_URLS = [
    # <- hier 2–5 echte FAQ-Unterseiten eintragen; Beispiele/Platzhalter:
    "https://faq-de.mediamarkt.ch/s/article/zahlungsmethoden-online",   # Beispiel: Zahlung
    "https://faq-de.mediamarkt.ch/s/article/rueckgabe-umtausch",        # Beispiel: Rückgabe
]

loader = WebBaseLoader(FAQ_URLS)
docs = loader.load()

print(f"Geladene Seiten: {len(docs)}")
print(docs[0].page_content[:400])


Geladene Seiten: 2



There is a an error with your request



There has been an error with your request.





Splitten

In [76]:
# Cell 4: Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=120)
splits = splitter.split_documents(docs)

len(splits), len(docs), sum(len(s.page_content) for s in splits) // max(1, len(splits))


(2, 2, 83)

Embeddings + FAISS

In [77]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# FAISS direkt aus Dokumenten bauen
vs = FAISS.from_documents(splits, embedding=embeddings)

print("FAISS Vektoren:", vs.index.ntotal)


FAISS Vektoren: 2


Prompt/Safeguard

In [78]:

import os
from dotenv import load_dotenv
load_dotenv()


from langchain_core.prompts import ChatPromptTemplate

SAFE_SYSTEM_PROMPT = """
Rolle & Aufgabe:
Du bist eine freundliche und sachliche Kundenberaterin des MediaMarkt Onlineshops (Schweiz).
Deine Aufgabe ist es, Kund:innen bei Fragen zu Produkten, Bestellungen, Rückgaben und allgemeinen Website-Themen zu helfen.

Verhaltensregeln (sehr wichtig):
1. Antworte ausschliesslich auf Basis der unten angegebenen Wissensquellen (Kontext).
2. Wenn im Kontext keine passende Information steht, sag höflich:
   "Dazu habe ich leider keine Informationen. Bitte wende dich direkt an den MediaMarkt Kundendienst."
3. Erfinde oder ergänze keine eigenen Fakten. Keine Spekulationen.
4. Bei rechtlichen oder Garantie-Themen zitiere nur Textstellen aus dem Kontext.
5. Antworte kurz, klar und freundlich auf Deutsch (Schweiz), maximal 3–5 Sätze.
6. Verwende neutrale, respektvolle Sprache und duze die Kund:innen konsequent.
7. Liste am Ende höchstens drei relevante Quellen unter der Überschrift "Quellen:" auf.

Kontextinformationen:
{context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SAFE_SYSTEM_PROMPT),
    ("human", "{question}")
])


Retriever + Antwort

In [79]:
def _retrieve(vs, query: str, k: int = 6):
    retriever = vs.as_retriever(
        search_type="mmr",
        search_kwargs={"k": k, "fetch_k": 24, "lambda_mult": 0.5},
    )
    # LC 0.1.x hat get_relevant_documents; LC 0.2.x nutzt invoke()
    try:
        return retriever.get_relevant_documents(query)  # 0.1.x
    except AttributeError:
        return retriever.invoke(query)                  # 0.2.x+

def answer_with_rag(question: str, k: int = 6) -> str:
    docs = _retrieve(vs, question, k=k)
    if not docs:
        return "Dazu habe ich keine gesicherten Infos in den FAQs."

    # Kontext + Quellen bauen
    context = "\n\n".join(d.page_content for d in docs[:k])
    seen, sources = set(), []
    for d in docs:
        src = d.metadata.get("source")
        if src and src not in seen:
            sources.append(src); seen.add(src)
        if len(sources) == 3:
            break

    # Prompt und LLM-Aufruf (achte auf den korrekten Import deiner Version)
    try:
        from langchain_core.prompts import ChatPromptTemplate
    except ModuleNotFoundError:
        from langchain_core.prompts import ChatPromptTemplate


    SAFE_SYSTEM_PROMPT = (
        "Du bist Kundenberater:in für MediaMarkt CH.\n"
        "Antworte NUR auf Basis des Kontexts. Wenn nichts passt, sag das offen.\n"
        "Kurz & präzise; am Ende max. 3 Quellen unter 'Quellen:'.\n\n"
        "Kontext:\n{context}"
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", SAFE_SYSTEM_PROMPT),
        ("human", "{question}")
    ])
    msgs = prompt.format_messages(context=context, question=question)
    res = llm.invoke(msgs).content

    if sources:
        res += "\n\nQuellen:\n" + "\n".join(f"- {s}" for s in sources)
    return res


Testfragen

In [80]:

print(answer_with_rag("Wie lange ist die Rückgabefrist?"))
print()
print(answer_with_rag("Welche Online-Zahlungsmethoden gibt es?"))
print()
print(answer_with_rag("Was deckt die Versicherung/Garantie ab?"))


Leider liegen mir im aktuellen Kontext keine Informationen zu den verfügbaren Zahlungsarten vor.  

**Quellen:** –

Quellen:
- https://faq-de.mediamarkt.ch/s/article/rueckgabe-umtausch
- https://faq-de.mediamarkt.ch/s/article/zahlungsmethoden-online
